<a href="https://colab.research.google.com/github/manojach87/miscellaneous/blob/master/coingecko.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
def readJsonLink(link):
  import urllib.request, json 
  with urllib.request.urlopen(link) as url:
      data = json.loads(url.read().decode())
      return data

In [90]:
marketsData=list()
for i in range(1,50):
  print(i)
  marketsData.append(readJsonLink("https://api.coingecko.com/api/v3/coins/markets?vs_currency=USD&order=market_cap_desc&per_page=250&page="+str(i)+"&sparkline=false"))
type(marketsData)
#marketsData

1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


list

In [91]:
import pandas as pd
marketsDf=pd.DataFrame()
for i in range(0,len(marketsData)):
  print(i)
  marketsDf=marketsDf.append(pd.DataFrame(marketsData[i]))

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48


In [93]:
marketsDf=marketsDf.reset_index()
#marketsDf[marketsDf.index==0]
marketsDf

,level_0,index,id,symbol,name,image,current_price,market_cap,market_cap_rank,fully_diluted_valuation,total_volume,high_24h,low_24h,price_change_24h,price_change_percentage_24h,market_cap_change_24h,market_cap_change_percentage_24h,circulating_supply,total_supply,max_supply,ath,ath_change_percentage,ath_date,atl,atl_change_percentage,atl_date,roi,last_updated
0,0,0,bitcoin,btc,Bitcoin,https://assets.coingecko.com/coins/images/1/la...,6.316700e+04,1.180631e+12,1,1.327119e+12,7.260682e+10,63693.00,60239.00,2094.070000,3.42882,4.613281e+10,4.06636,1.868202e+07,2.100000e+07,2.100000e+07,63693.00,-0.78025,2021-04-14T00:09:07.264Z,67.810000,9.309726e+04,2013-07-06T00:00:00.000Z,None,2021-04-14T02:46:28.890Z
1,1,1,ethereum,eth,Ethereum,https://assets.coingecko.com/coins/images/279/...,2.289290e+03,2.640379e+11,2,NaN,3.614582e+10,2313.49,2156.20,117.410000,5.40571,1.400172e+10,5.59988,1.154598e+08,NaN,NaN,2313.49,-1.15211,2021-04-13T18:29:28.394Z,0.432979,5.280642e+05,2015-10-20T00:00:00.000Z,"{'times': 47.44844714644223, 'currency': 'btc'...",2021-04-14T02:46:52.958Z
2,2,2,binancecoin,bnb,Binance Coin,https://assets.coingecko.com/coins/images/825/...,5.610900e+02,8.676804e+10,3,9.575178e+10,1.055811e+10,580.82,536.97,-14.754752,-2.56227,-2.912194e+09,-3.24731,1.545337e+08,1.705337e+08,1.705337e+08,610.06,-7.96194,2021-04-12T09:32:32.635Z,0.039818,1.410035e+06,2017-10-19T00:00:00.000Z,None,2021-04-14T02:46:10.940Z
3,3,3,ripple,xrp,XRP,https://assets.coingecko.com/coins/images/44/l...,1.790000e+00,8.265175e+10,4,NaN,2.834679e+10,1.88,1.43,0.345961,23.89967,1.622739e+10,24.42987,4.597332e+10,1.000000e+11,NaN,3.40,-47.09882,2018-01-07T00:00:00.000Z,0.002686,6.682784e+04,2014-05-22T00:00:00.000Z,None,2021-04-14T02:46:44.084Z
4,4,4,cardano,ada,Cardano,https://assets.coingecko.com/coins/images/975/...,1.450000e+00,4.637784e+10,5,NaN,6.029396e+09,1.45,1.29,0.135671,10.32649,4.338983e+09,10.32136,3.203205e+10,4.500000e+10,NaN,1.48,-1.99625,2021-03-18T15:55:08.465Z,0.019253,7.420263e+03,2020-03-13T02:22:55.044Z,None,2021-04-14T02:45:16.835Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6678,6678,178,apeusd-aave-synthetic-usd-dec-2021,apeUSD-AAVE-DEC21,apeUSD-AAVE Synthetic USD (Dec 2021),https://assets.coingecko.com/coins/images/1439...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.400000e+03,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6679,6679,179,taleshcoin,talc,Taleshcoin,https://assets.coingecko.com/coins/images/1306...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.002000e+11,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6680,6680,180,wrapped-huobi-token,wht,Wrapped Huobi Token,https://assets.coingecko.com/coins/images/1404...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.276628e+07,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None
6681,6681,181,instadapp,inst,Instadapp,https://assets.coingecko.com/coins/images/1468...,NaN,NaN,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.000000e+08,NaN,NaN,0.00000,None,NaN,0.000000e+00,None,None,None


In [95]:
len(marketsDf)

6683

In [103]:
btcSupply=marketsDf[marketsDf.symbol=="btc"].circulating_supply[0]
btcSupply

18682018.0

In [108]:
marketsDf["btc_relative_val"]=marketsDf.circulating_supply/btcSupply*marketsDf.current_price

In [113]:
marketsDf.sort_values(by=["btc_relative_val"], ascending=[0], inplace=True)

In [117]:
marketsDf[["symbol","btc_relative_val","current_price"]]

,symbol,btc_relative_val,current_price
0,btc,63167.000000,6.316700e+04
1,eth,14148.421078,2.289290e+03
2,bnb,4641.216315,5.610900e+02
3,xrp,4404.890270,1.790000e+00
4,ada,2486.159415,1.450000e+00
...,...,...,...
6678,apeUSD-AAVE-DEC21,NaN,NaN
6679,talc,NaN,NaN
6680,wht,NaN,NaN
6681,inst,NaN,NaN
